In [22]:
import tensorflow as tf
import os
import sklearn
#tf.enable_eager_execution()

# MNIST - выборка для обучения

Загрузим выборку для обучния, как в предыдущем занятии

In [2]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

X, y = mnist["data"], mnist["target"]
X.shape, y.shape

((70000, 784), (70000,))

In [3]:
X_train, y_train,X_test,  y_test = X[:60000], y[:60000], X[60000:], y[60000:]

# Tesorflow - low level

# Tensorflow - high level

In [4]:
tf.reset_default_graph()
X = X_train
y = y_train
datasetX = tf.data.Dataset.from_tensor_slices(X)
datasety = tf.data.Dataset.from_tensor_slices(y)
print (X.shape, y.shape)

(60000, 784) (60000,)


In [ ]:
Сделаем два классификатора

In [6]:
nums = tf.feature_column.numeric_column("image", (784,))


estimator3 = tf.estimator.DNNClassifier(
    feature_columns=[nums],
    n_classes=10,
    hidden_units=[1024, 512, 256],
    model_dir=os.path.expanduser("~/model/dnn_mnist_3")
)

estimator2 = tf.estimator.DNNClassifier(
    feature_columns=[nums],
    n_classes=10,
    hidden_units=[300,100],
    model_dir=os.path.expanduser("~/model/dnn_mnist_2")
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_is_chief': True, '_save_summary_steps': 100, '_task_id': 0, '_save_checkpoints_steps': None, '_model_dir': '/home/student/model/dnn_mnist_3', '_global_id_in_cluster': 0, '_service': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_num_worker_replicas': 1, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f00376cd6d8>, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_master': '', '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_is_chief': True, '_save_summary_steps': 100, '_task_id': 0, '_save_checkpoints_steps': None, '_model_dir': '/home/student/model/dnn_mnist_2', '_global_id_in_cluster': 0, '_service': None, '_num_ps_replicas': 0, '_evaluation_master': '', '_num_worker_

In [7]:
def input_evaluation_set():
    dataset = tf.data.Dataset.from_tensor_slices(({'image':X}, y.astype(int)))
    return dataset.shuffle(1000).repeat().batch(100)

In [8]:
estimator2.train(
    input_fn=lambda:input_evaluation_set())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/student/model/dnn_mnist_2/model.ckpt-464993
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 464994 into /home/student/model/dnn_mnist_2/model.ckpt.
INFO:tensorflow:loss = 0.14459193, step = 464994
INFO:tensorflow:global_step/sec: 238.911
INFO:tensorflow:loss = 0.15878722, step = 465094 (0.426 sec)
INFO:tensorflow:global_step/sec: 243.093
INFO:tensorflow:loss = 0.11293854, step = 465194 (0.412 sec)
INFO:tensorflow:global_step/sec: 244.41
INFO:tensorflow:loss = 0.11579496, step = 465294 (0.408 sec)
INFO:tensorflow:global_step/sec: 241.551
INFO:tensorflow:loss = 0.019358668, step = 465394 (0.412 sec)
INFO:tensorflow:global_step/sec: 244.204
INFO:tensorflow:loss = 0.30283362, step = 465494 (0.411 sec)
INFO:tensorflow:g

KeyboardInterrupt: 

In [10]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [12]:
estimator2.evaluate(input_fn=lambda: eval_input_fn({'image':X_test}, y_test.astype(int), 100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-12:19:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/student/model/dnn_mnist_2/model.ckpt-464994
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-13-12:19:42
INFO:tensorflow:Saving dict for global step 464994: accuracy = 0.9513, average_loss = 0.88204306, global_step = 464994, loss = 88.20431


{'accuracy': 0.9513,
 'average_loss': 0.88204306,
 'global_step': 464994,
 'loss': 88.20431}

Мы получили классификатор, который работает с точность 0.95

In [ ]:
estimator3.train(
    input_fn=lambda:input_evaluation_set())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /home/borsh/model/dnn_mnist_3/model.ckpt.
INFO:tensorflow:loss = 22584.902, step = 1
INFO:tensorflow:global_step/sec: 52.4678
INFO:tensorflow:loss = 0.0, step = 101 (1.909 sec)
INFO:tensorflow:global_step/sec: 54.8295
INFO:tensorflow:loss = 22.575281, step = 201 (1.818 sec)
INFO:tensorflow:global_step/sec: 44.6533
INFO:tensorflow:loss = 132.54565, step = 301 (2.244 sec)
INFO:tensorflow:global_step/sec: 62.1332
INFO:tensorflow:loss = 1.6811857, step = 401 (1.605 sec)
INFO:tensorflow:global_step/sec: 40.6912
INFO:tensorflow:loss = 0.69007, step = 501 (2.457 sec)
INFO:tensorflow:global_step/sec: 31.3323
INFO:tensorflow:loss = 1500.1787, step = 601 (3.191 sec)
INFO:tensorflow:global_step/sec: 59.1316
INFO:tensor

INFO:tensorflow:loss = 1.2403712, step = 8001 (1.559 sec)
INFO:tensorflow:global_step/sec: 49.1922
INFO:tensorflow:loss = 30.906475, step = 8101 (2.032 sec)
INFO:tensorflow:global_step/sec: 49.6761
INFO:tensorflow:loss = 0.013246907, step = 8201 (2.012 sec)
INFO:tensorflow:global_step/sec: 57.3993
INFO:tensorflow:loss = 0.08086318, step = 8301 (1.743 sec)
INFO:tensorflow:global_step/sec: 57.6462
INFO:tensorflow:loss = 739.6668, step = 8401 (1.733 sec)
INFO:tensorflow:global_step/sec: 40.1712
INFO:tensorflow:loss = 1.0596805, step = 8501 (2.492 sec)
INFO:tensorflow:global_step/sec: 50.703
INFO:tensorflow:loss = 2.6117003, step = 8601 (1.971 sec)
INFO:tensorflow:global_step/sec: 55.4507
INFO:tensorflow:loss = 6.9641275, step = 8701 (1.803 sec)
INFO:tensorflow:global_step/sec: 49.3825
INFO:tensorflow:loss = 0.044486985, step = 8801 (2.027 sec)
INFO:tensorflow:global_step/sec: 42.8814
INFO:tensorflow:loss = 1.43205, step = 8901 (2.330 sec)
INFO:tensorflow:global_step/sec: 58.2441
INFO:tens

INFO:tensorflow:global_step/sec: 60.0341
INFO:tensorflow:loss = 0.00038521853, step = 16301 (1.666 sec)
INFO:tensorflow:global_step/sec: 62.6768
INFO:tensorflow:loss = 2.3340487, step = 16401 (1.596 sec)
INFO:tensorflow:global_step/sec: 64.5956
INFO:tensorflow:loss = 16.63204, step = 16501 (1.546 sec)
INFO:tensorflow:global_step/sec: 64.4796
INFO:tensorflow:loss = 0.040368155, step = 16601 (1.551 sec)
INFO:tensorflow:global_step/sec: 65.0629
INFO:tensorflow:loss = 2.3034906, step = 16701 (1.537 sec)
INFO:tensorflow:global_step/sec: 64.2099
INFO:tensorflow:loss = 42.479683, step = 16801 (1.560 sec)
INFO:tensorflow:global_step/sec: 65.3687
INFO:tensorflow:loss = 0.41934657, step = 16901 (1.530 sec)
INFO:tensorflow:global_step/sec: 58.9128
INFO:tensorflow:loss = 7.5619755, step = 17001 (1.696 sec)
INFO:tensorflow:global_step/sec: 65.2118
INFO:tensorflow:loss = 7.1833043, step = 17101 (1.534 sec)
INFO:tensorflow:global_step/sec: 63.9981
INFO:tensorflow:loss = 0.16895233, step = 17201 (1.56

INFO:tensorflow:global_step/sec: 60.5331
INFO:tensorflow:loss = 0.30750018, step = 24501 (1.654 sec)
INFO:tensorflow:global_step/sec: 63.3582
INFO:tensorflow:loss = 27.196709, step = 24601 (1.576 sec)
INFO:tensorflow:global_step/sec: 63.127
INFO:tensorflow:loss = 1.7974294, step = 24701 (1.583 sec)
INFO:tensorflow:global_step/sec: 62.4127
INFO:tensorflow:loss = 1.60977, step = 24801 (1.603 sec)
INFO:tensorflow:global_step/sec: 64.3873
INFO:tensorflow:loss = 9.403481, step = 24901 (1.554 sec)
INFO:tensorflow:global_step/sec: 61.8542
INFO:tensorflow:loss = 0.19987571, step = 25001 (1.615 sec)
INFO:tensorflow:global_step/sec: 63.3072
INFO:tensorflow:loss = 1.1089252, step = 25101 (1.581 sec)
INFO:tensorflow:global_step/sec: 51.5031
INFO:tensorflow:loss = 29.817, step = 25201 (1.940 sec)
INFO:tensorflow:global_step/sec: 63.1308
INFO:tensorflow:loss = 0.004797592, step = 25301 (1.586 sec)
INFO:tensorflow:global_step/sec: 64.6186
INFO:tensorflow:loss = 0.43036088, step = 25401 (1.546 sec)
IN

INFO:tensorflow:global_step/sec: 62.9982
INFO:tensorflow:loss = 8.839984, step = 32701 (1.586 sec)
INFO:tensorflow:global_step/sec: 64.4154
INFO:tensorflow:loss = 0.05087463, step = 32801 (1.553 sec)
INFO:tensorflow:global_step/sec: 62.5725
INFO:tensorflow:loss = 0.6238855, step = 32901 (1.598 sec)
INFO:tensorflow:global_step/sec: 64.2898
INFO:tensorflow:loss = 7.617551, step = 33001 (1.558 sec)
INFO:tensorflow:global_step/sec: 61.8779
INFO:tensorflow:loss = 0.051360827, step = 33101 (1.614 sec)
INFO:tensorflow:global_step/sec: 63.4293
INFO:tensorflow:loss = 0.27949154, step = 33201 (1.577 sec)
INFO:tensorflow:global_step/sec: 58.8865
INFO:tensorflow:loss = 6.9756665, step = 33301 (1.697 sec)
INFO:tensorflow:global_step/sec: 63.4991
INFO:tensorflow:loss = 0.52490765, step = 33401 (1.576 sec)
INFO:tensorflow:global_step/sec: 63.8257
INFO:tensorflow:loss = 0.063022956, step = 33501 (1.566 sec)
INFO:tensorflow:global_step/sec: 63.2362
INFO:tensorflow:loss = 22.94643, step = 33601 (1.581 s

...

INFO:tensorflow:global_step/sec: 56.5021
INFO:tensorflow:loss = 0.0017030027, step = 192901 (1.771 sec)
INFO:tensorflow:global_step/sec: 55.2114
INFO:tensorflow:loss = 0.003992658, step = 193001 (1.810 sec)
INFO:tensorflow:global_step/sec: 56.0194
INFO:tensorflow:loss = 0.0022013306, step = 193101 (1.787 sec)
INFO:tensorflow:global_step/sec: 54.4725
INFO:tensorflow:loss = 0.0016104232, step = 193201 (1.838 sec)
INFO:tensorflow:global_step/sec: 53.9648
INFO:tensorflow:loss = 0.008536357, step = 193301 (1.850 sec)
INFO:tensorflow:global_step/sec: 40.2586
INFO:tensorflow:loss = 0.0018323493, step = 193401 (2.483 sec)
INFO:tensorflow:global_step/sec: 55.7731
INFO:tensorflow:loss = 0.0006181006, step = 193501 (1.796 sec)
INFO:tensorflow:global_step/sec: 54.6677
INFO:tensorflow:loss = 0.0043795374, step = 193601 (1.830 sec)
INFO:tensorflow:global_step/sec: 52.4984
INFO:tensorflow:loss = 0.0031411787, step = 193701 (1.904 sec)
INFO:tensorflow:global_step/sec: 53.442
INFO:tensorflow:loss = 0.0

INFO:tensorflow:loss = 0.0015302699, step = 200801 (1.859 sec)
INFO:tensorflow:global_step/sec: 53.4317
INFO:tensorflow:loss = 0.0032050954, step = 200901 (1.870 sec)
INFO:tensorflow:global_step/sec: 55.6267
INFO:tensorflow:loss = 2.6821841e-05, step = 201001 (1.796 sec)
INFO:tensorflow:global_step/sec: 54.8437
INFO:tensorflow:loss = 0.00017927798, step = 201101 (1.824 sec)
INFO:tensorflow:global_step/sec: 53.9523
INFO:tensorflow:loss = 0.00048051804, step = 201201 (1.854 sec)
INFO:tensorflow:global_step/sec: 45.2628
INFO:tensorflow:loss = 0.0019438876, step = 201301 (2.209 sec)
INFO:tensorflow:global_step/sec: 55.0881
INFO:tensorflow:loss = 0.0028605168, step = 201401 (1.818 sec)
INFO:tensorflow:global_step/sec: 55.4524
INFO:tensorflow:loss = 0.004264069, step = 201501 (1.801 sec)
INFO:tensorflow:global_step/sec: 53.4467
INFO:tensorflow:loss = 0.00040693948, step = 201601 (1.870 sec)
INFO:tensorflow:global_step/sec: 54.6998
INFO:tensorflow:loss = 0.005770254, step = 201701 (1.829 sec)

INFO:tensorflow:loss = 0.00048073631, step = 208601 (1.721 sec)
INFO:tensorflow:global_step/sec: 61.1747
INFO:tensorflow:loss = 0.0058945986, step = 208701 (1.635 sec)
INFO:tensorflow:global_step/sec: 63.5472
INFO:tensorflow:loss = 0.0029788413, step = 208801 (1.575 sec)
INFO:tensorflow:global_step/sec: 60.2233
INFO:tensorflow:loss = 2.3960938e-05, step = 208901 (1.660 sec)
INFO:tensorflow:global_step/sec: 59.7029
INFO:tensorflow:loss = 0.0019308028, step = 209001 (1.674 sec)
INFO:tensorflow:global_step/sec: 61.5926
INFO:tensorflow:loss = 0.0011082083, step = 209101 (1.624 sec)
INFO:tensorflow:global_step/sec: 62.7586
INFO:tensorflow:loss = 0.0043219416, step = 209201 (1.595 sec)
INFO:tensorflow:global_step/sec: 64.4057
INFO:tensorflow:loss = 0.006608203, step = 209301 (1.553 sec)
INFO:tensorflow:global_step/sec: 63.478
INFO:tensorflow:loss = 0.0033100862, step = 209401 (1.573 sec)
INFO:tensorflow:global_step/sec: 60.7755
INFO:tensorflow:loss = 0.007876309, step = 209501 (1.646 sec)
IN

INFO:tensorflow:loss = 0.003800054, step = 216501 (2.321 sec)
INFO:tensorflow:global_step/sec: 56.0183
INFO:tensorflow:loss = 0.00017988386, step = 216601 (1.785 sec)
INFO:tensorflow:global_step/sec: 56.0037
INFO:tensorflow:loss = 0.0026473976, step = 216701 (1.786 sec)
INFO:tensorflow:global_step/sec: 55.1636
INFO:tensorflow:loss = 0.002276691, step = 216801 (1.812 sec)
INFO:tensorflow:global_step/sec: 55.0982
INFO:tensorflow:loss = 0.0020209805, step = 216901 (1.817 sec)
INFO:tensorflow:global_step/sec: 55.4722
INFO:tensorflow:loss = 0.001249168, step = 217001 (1.799 sec)
INFO:tensorflow:global_step/sec: 56.242
INFO:tensorflow:loss = 0.003800317, step = 217101 (1.782 sec)
INFO:tensorflow:global_step/sec: 54.9369
INFO:tensorflow:loss = 0.00078944047, step = 217201 (1.816 sec)
INFO:tensorflow:global_step/sec: 52.9711
INFO:tensorflow:loss = 0.00031968174, step = 217301 (1.889 sec)
INFO:tensorflow:global_step/sec: 53.6085
INFO:tensorflow:loss = 0.0021321794, step = 217401 (1.865 sec)
INF

INFO:tensorflow:loss = 0.0003932227, step = 224401 (2.258 sec)
INFO:tensorflow:global_step/sec: 53.7298
INFO:tensorflow:loss = 0.0013619071, step = 224501 (1.859 sec)
INFO:tensorflow:global_step/sec: 55.5457
INFO:tensorflow:loss = 0.006302981, step = 224601 (1.801 sec)
INFO:tensorflow:global_step/sec: 54.9542
INFO:tensorflow:loss = 0.0018944907, step = 224701 (1.820 sec)
INFO:tensorflow:global_step/sec: 54.5009
INFO:tensorflow:loss = 0.0032083013, step = 224801 (1.835 sec)
INFO:tensorflow:global_step/sec: 54.955
INFO:tensorflow:loss = 0.008327376, step = 224901 (1.821 sec)
INFO:tensorflow:global_step/sec: 55.1818
INFO:tensorflow:loss = 0.002588917, step = 225001 (1.811 sec)
INFO:tensorflow:global_step/sec: 53.467
INFO:tensorflow:loss = 0.0013782098, step = 225101 (1.870 sec)
INFO:tensorflow:global_step/sec: 56.1142
INFO:tensorflow:loss = 0.002088042, step = 225201 (1.781 sec)
INFO:tensorflow:global_step/sec: 54.6736
INFO:tensorflow:loss = 0.00080268213, step = 225301 (1.831 sec)
INFO:t

In [13]:
estimator3.evaluate(input_fn=lambda: eval_input_fn({'image':X_test}, y_test.astype(int), 100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-12:20:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/student/model/dnn_mnist_3/model.ckpt-208295
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-13-12:20:19
INFO:tensorflow:Saving dict for global step 208295: accuracy = 0.9586, average_loss = 0.5312688, global_step = 208295, loss = 53.12688


{'accuracy': 0.9586,
 'average_loss': 0.5312688,
 'global_step': 208295,
 'loss': 53.12688}

Точность чуть больше, но для тренировки требуется существенно большее время

# Задание

Необходимо сделать классификатор на базе данных iris и оценить его точность. 
Сделать две сети: hidden_uints = [1024, 512, 256], и hidden_uints=[300,100]

Загрузка данных

In [26]:
from sklearn.datasets import load_iris
import sklearn.model_selection
iris = load_iris()
X = iris.data
y = iris.target
print (X.shape, y.shape)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
print (X_train.shape,  y_train.shape, X_test.shape, y_test.shape)

(150, 4) (150,)
(112, 4) (112,) (38, 4) (38,)
